In [1]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade numpy opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
pip install pygame


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk
import os
import numpy as np
import cv2 as cv
import uuid
from tkinter import font
import pygame

class Window(Frame):
    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.master = master
        self.pos = []
        self.master.title("Coloring Black and White Images")
        self.pack(fill=BOTH, expand=1)
        self.master.configure(bg="skyblue")
        
        
        menu = Menu(self.master)
        self.master.config(menu=menu)

        file = Menu(menu)
        file.add_command(label="Upload Image", command=self.uploadImage)
        file.add_command(label="Clear Canvas", command=self.clear_canvas)
        menu.add_cascade(label="File", menu=file)
          

        self.canvas_bw = tk.Canvas(self,bg="ivory2")
        self.canvas_bw.pack(side=LEFT, fill=tk.BOTH, expand=True)

        self.canvas_colored = tk.Canvas(self,bg="ivory2")
        self.canvas_colored.pack(side=RIGHT, fill=tk.BOTH, expand=True)

        self.image_bw = None
        self.image_colored = None
        self.image_path = None        
        
        label1 = Label(self)
        label1.place(x=400, y=370)

        
        # Add a Color Image button
        self.color_button = Button(self, text="Color Image", command=self.color, bg="plum1", fg="black", activebackground="plum1", activeforeground="white")
        self.color_button.place(x=300,y=650,height=45)
        self.color_button['font'] = myFont

        # Add a Download Image button
        self.download_button = Button(self, text="Download Image", command=self.download_image,bg="lightgreen", fg="black", activebackground="plum1", activeforeground="white")
        self.download_button.place(x=1070,y=650,height=45)
        self.download_button['font'] = myFont
        
    def play_sound(self, sound_file):
        pygame.mixer.Sound(sound_file).play()
        
        
    def clear_canvas(self):
        clear_sound.play()  
        self.canvas_bw.delete("all")
        self.canvas_colored.delete("all")

    def uploadImage(self):
        self.clear_canvas()
        filename = filedialog.askopenfilename(initialdir=os.getcwd())
        if not filename:
            return
        self.image_path = filename
        load = Image.open(filename)

        load = load.resize((800, 600), Image.Resampling.LANCZOS)

        w, h = load.size
        self.render_bw = ImageTk.PhotoImage(load)
        self.image_bw = self.canvas_bw.create_image((w / 2, h / 2), image=self.render_bw)
        self.play_sound('mouse-click-153941.mp3')
        
    def color(self):
        download_sound.play()
        if self.image_path is None:
            return

        net = cv.dnn.readNetFromCaffe('colorization_deploy_v2.prototxt', 'colorization_release_v2.caffemodel')
        pts = np.load('pts_in_hull.npy')

        class8 = net.getLayerId("class8_ab")
        conv8 = net.getLayerId("conv8_313_rh")
        pts = pts.transpose().reshape(2, 313, 1, 1)

        net.getLayer(class8).blobs = [pts.astype("float32")]
        net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype='float32')]

        image = cv.imread(self.image_path)
        scaled = image.astype("float32") / 255.0
        lab = cv.cvtColor(scaled, cv.COLOR_BGR2LAB)

        resized = cv.resize(lab, (224, 224))
        L = cv.split(resized)[0]
        L -= 50

        net.setInput(cv.dnn.blobFromImage(L))
        ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

        ab = cv.resize(ab, (image.shape[1], image.shape[0]))

        L = cv.split(lab)[0]
        colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

        colorized = cv.cvtColor(colorized, cv.COLOR_LAB2BGR)
        colorized = np.clip(colorized, 0, 1)

        colorized = (255 * colorized).astype("uint8")

        self.output_file = f"output_{str(uuid.uuid4())}.png"
        cv.imwrite(self.output_file, colorized)

        load = Image.open(self.output_file)
        load = load.resize((800, 600), Image.Resampling.LANCZOS)

        w, h = load.size
        self.render_colored = ImageTk.PhotoImage(load)
        self.image_colored = self.canvas_colored.create_image((w / 2, h / 2), image=self.render_colored)

    def download_image(self):
        download_sound.play()
        if self.output_file:
            save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
            if save_path:
                os.rename(self.output_file, save_path)
    

    
def start_colorization():
    start_sound.play()
    start_button.place(x=550, y=350, width=1000, height=850)
    start_button['font'] = myFont # Adjust the font if needed
    start_button.destroy()
    open_colorization_window()


def open_colorization_window():
    global app
    root.geometry("%dx%d" % (1980, 1000))
    app = Window(root)
    app.pack(fill=tk.BOTH, expand=1)
    root.protocol("WM_DELETE_WINDOW", on_closing)

def on_closing():
    pygame.quit()
    root.destroy()

pygame.init()
start_sound = pygame.mixer.Sound(r'mouse-click-153941.mp3')
pygame.init()
clear_sound = pygame.mixer.Sound(r'mouse-click-153941.mp3')
pygame.init()
download_sound = pygame.mixer.Sound(r'mouse-click-153941.mp3')
pygame.init()
color_sound = pygame.mixer.Sound(r'mouse-click-153941.mp3')

root = tk.Tk()
root.geometry("%dx%d" % (300, 200))
root.title("B&W Image Colorization Start Page")
#root.configure(bg="skyblue")

myFont = font.Font(size=15)
myFont1 = font.Font(size=25)

background_image = Image.open("BG.png")
desired_width = 1515
desired_height = 900
background_image = background_image.resize((desired_width, desired_height), Image.Resampling.LANCZOS)

background_photo = ImageTk.PhotoImage(background_image)
background_label = Label(root, image=background_photo)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

start_button = Button(root, text="Start", command=start_colorization, bg='burlywood4')
start_button.place(x=200, y=600)
start_button['font'] = myFont1

root.protocol("WM_DELETE_WINDOW", on_closing)

root.mainloop()